In [4]:
import numpy as np
import random

In [5]:
class AntColonyOptimizer:
    def __init__(self, connections, antNum, cityNum, alpha, beta, evRate, maxTime, cityNames, costWeight):
        self.connections = connections
        self.antNum = antNum
        self.cityNum = cityNum
        self.alpha = alpha
        self.beta = beta
        self.evRate = evRate
        self.maxTime = maxTime
        self.cityNames = cityNames
        self.costWeight = costWeight
        self.times, self.costs = self.CalcAll()

    def CalcAll(self):
        times = np.zeros((self.cityNum, self.cityNum))
        costs = np.zeros((self.cityNum, self.cityNum))
        for conn in self.connections:
            times[conn[0], conn[1]] = conn[2]
            times[conn[1], conn[0]] = conn[2]
            costs[conn[0], conn[1]] = conn[3]
            costs[conn[1], conn[0]] = conn[3]
        return times, costs

    def Run(self, iterationNum):
      pheromoneMatrix = np.ones((self.cityNum, self.cityNum))
      visibility = self.CalcVis()
    
      bestPath = None
      bestTime = float('inf')
      bestCost = float('inf')

      for _ in range(iterationNum):
          allPaths = []
          allTimes = []
          allCosts = []
          for _ in range(self.antNum):
              path, time, cost = self.PathGen(pheromoneMatrix)
              allPaths.append(path)
              allTimes.append(time)
              allCosts.append(cost)

          pheromoneMatrix = self.UpdatePheromones(pheromoneMatrix, allPaths, allTimes, allCosts)

          minIndex = np.argmin(allTimes)
          minTime = allTimes[minIndex]
          minCost = allCosts[minIndex]
          if minTime < bestTime:
              bestPath = allPaths[minIndex]
              bestTime = minTime
              bestCost = minCost

          if bestTime <= self.maxTime:
              break

      return bestPath, bestTime, bestCost


    def SolGen(self, pheromoneMatrix):
        allPaths = []
        allCosts = []

        for _ in range(self.antNum):
            path, cost = self.PathGen(pheromoneMatrix)
            allPaths.append(path)
            allCosts.append(cost)

        return allPaths, allCosts

    def PathGen(self, pheromoneMatrix):
      visited = [np.random.randint(self.cityNum)]
      movesLim = 17

      for _ in range(movesLim - 1):
          nextCity = self.PickNext(visited[-1], visited, pheromoneMatrix)
          visited.append(nextCity)
        
          # Check if all cities have been visited
          if len(set(visited)) == self.cityNum:
              break

      totalTime = sum(self.times[visited[i], visited[i + 1]] for i in range(len(visited) - 1)) + self.times[visited[-1], visited[0]]
      totalCost = sum(self.costs[visited[i], visited[i + 1]] for i in range(len(visited) - 1)) + self.costs[visited[-1], visited[0]]
    
      return visited, totalTime, totalCost

    def CalcVis(self):
      visibility = np.zeros((self.cityNum, self.cityNum))
      for i in range(self.cityNum):
          for j in range(self.cityNum):
              if i != j:
                  visibility[i, j] = 1 / self.times[i, j]
      return visibility


    def PickNext(self, currentCity, visited, pheromoneMatrix):
      probabilities = []
      unvCitiesWeight = 650  # Increase this value to prioritize visiting unvisited cities

      for city in range(self.cityNum):
          if self.times[currentCity, city] == 0:
              probabilities.append(0)
          else:
              cityWeight = unvCitiesWeight if city not in visited else 1
              probabilities.append(
                  (pheromoneMatrix[currentCity, city] ** self.alpha) * ((1 / self.times[currentCity, city]) ** self.beta) * cityWeight
              )

      probSum = sum(probabilities)

      if probSum == 0:
          # If probSum is zero, choose the next city uniformly at random
          return np.random.randint(self.cityNum)
      else:
          probabilities = [prob / probSum for prob in probabilities]
          return np.random.choice(self.cityNum, p=probabilities)


    def UpdatePheromones(self, pheromoneMatrix, allPaths, allTimes, allCosts):
        pheromoneMatrix *= (1 - self.evRate)

        for path, time, cost in zip(allPaths, allTimes, allCosts):
            weighVal = time + self.costWeight * cost
            for i in range(len(path) - 1):
                pheromoneMatrix[path[i], path[i+1]] += 1 / weighVal

        return pheromoneMatrix

In [6]:
def main():
    connections = [
         (0, 1, 136 / 60, 98),
         (1, 2, 82 / 60, 80),
         (1, 6, 480 / 60, 345),
         (1, 9, 112 / 60, 185),
         (1, 11, 225 / 60, 380),
         (1, 10, 390 / 60, 400),
         (2, 3, 105 / 60, 48),
         (3, 4, 120 / 60, 40),
         (3, 5, 364 / 60, 235),
         (4, 6, 120 / 60, 40),
         (5, 6, 232 / 60, 125),
         (6, 7, 464 / 60, 240),
         (7, 8, 168 / 60, 125),
         (7, 9, 174 / 60, 180),
         (9, 10, 200 / 60, 320),
         (10, 11, 150 / 60, 98),
    ]

    cityNames = [
        "London", "Paris", "Brussels", "Amsterdam", "Cologne",
        "Berlin", "Frankfurt", "Milan", "Rome", "Lyon",
        "Barcelona", "Madrid"
    ]

    cityNum = len(cityNames)
    aco = AntColonyOptimizer(
      connections=connections,
      antNum=1000,
      cityNum=cityNum,
      alpha=1,
      beta=5,
      evRate=0.1,
      maxTime=72,
      cityNames=cityNames,
      costWeight=0.5  # Change this value to adjust the trade-off
    )

    bestPath, bestTime, bestCost = aco.Run(iterationNum=1000000)

    if bestTime > 72:
        raise Exception("No solution found within the 72-hour limit.")
    else:
        bestPath_names = [cityNames[i] for i in bestPath]
        print("Best path found:", bestPath_names)
        print("Time:", bestTime, "hours")
        print("Cost:", bestCost)

main()

<ipython-input-5-99ed78a3bddb>:91: RuntimeWarning: divide by zero encountered in double_scalars
  visibility[i, j] = 1 / self.times[i, j]


Best path found: ['Madrid', 'Barcelona', 'Lyon', 'Paris', 'Brussels', 'Amsterdam', 'Cologne', 'Frankfurt', 'Cologne', 'Amsterdam', 'Brussels', 'Paris', 'London', 'Paris', 'Brussels', 'Paris', 'Brussels']
Time: 30.56666666666667 hours
Cost: 1455.0
